In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import pyprind

import tarfile

# Use the correct file name
tar = tarfile.open("/Users/suarezcruzrosy/Desktop/aclImdb_v1.tar")
tar.extractall()
tar.close()


/var/folders/s3/jrfb_5c518bfrqk54gkg37kw0000gn/T/ipykernel_9827/790394354.py:11: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall()


In [18]:
basepath = '/Users/suarezcruzrosy/Desktop/aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as f:
                txt = f.read()
            df = pd.concat([df, pd.DataFrame([[txt, labels[l]]], columns=['review', 'sentiment'])], ignore_index=True)
            pbar.update()



In [19]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [20]:
df = pd.read_csv('movie_data.csv', encoding= 'utf-8') 
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [22]:
df.shape
(50000, 2)

(50000, 2)